In [28]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

In [29]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('Interviewdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(questions))
with open('Interviewdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(answers))
with open('Interviewdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    labels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(labels))

with open('Interviewtestdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testquestions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testquestions))

with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers))  

408
408
408
4
4


In [30]:
completedata = questions.copy()
for i in answers:
    completedata.append(i)

In [31]:
len(completedata)

816

In [32]:
len(questions)

408

In [33]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [34]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [35]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [36]:
for i, sent in enumerate(questions):
    questions[i] = [w if w in word_to_index else unknown_token for w in sent]

In [37]:
for i, sent in enumerate(answers):
    answers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [38]:
dummy = 'a'

In [39]:
for i, sent in enumerate(testquestions):
    testquestions[i] = [w if w in word_to_index else dummy for w in sent]

In [40]:
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else dummy for w in sent]

In [41]:
import numpy as np
X_question = np.asarray([[word_to_index[w] for w in sent] for sent in questions])
y_answer = np.asarray([[word_to_index[w] for w in sent] for sent in answers])

In [42]:
x_example, y_example = X_question[1], y_answer[1]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
in a program the code is for file opening , reading each unit of record , processing of recode one by one then closing of the file . in which block closing of file statement should be coded .
[5, 8, 184, 0, 72, 2, 19, 47, 1570, 3, 486, 216, 1052, 6, 1322, 3, 370, 6, 1684, 60, 26, 60, 104, 739, 6, 0, 47, 1, 5, 34, 186, 739, 6, 47, 213, 96, 15, 584, 1]

y:
finally block , as in case any exception is raised then also file will be surely closed .
[878, 186, 3, 23, 5, 149, 73, 82, 2, 518, 104, 163, 47, 20, 15, 1574, 638, 1]


In [43]:
X_testquestion = np.asarray([[word_to_index[w] for w in sent] for sent in testquestions])
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

In [44]:
questionlength = [len(i) for i in X_question]
answerlength = [len(i) for i in y_answer]

In [45]:
testquestionlength = [len(i) for i in X_testquestion]
testanswerlength = [len(i) for i in y_testanswer]

In [46]:
import numpy as np
x = X_question
y = y_answer
max_length = max(len(row) for row in y)
x_padded = np.array([row + [0] * (max_length - len(row)) for row in x])
y_padded = np.array([row + [0] * (max_length - len(row)) for row in y])

p = X_testquestion
q = y_testanswer
x_testpadded = np.array([row + [0] * (max_length - len(row)) for row in p])
y_testpadded = np.array([row + [0] * (max_length - len(row)) for row in q])

In [47]:
len(x_padded), len(y_padded), len(x_padded[1]), len(y_padded[1])

(408, 408, 164, 164)

In [48]:
len(x_testpadded), len(y_testpadded), len(x_testpadded[1]), len(y_testpadded[1])

(4, 4, 164, 164)

In [49]:
len(wordlist), len(embeddings)

(1719, 1719)

In [50]:
tf.InteractiveSession().run(tf.concat([[[1, 2, 3], [4, 5, 6]],[[7, 8, 9], [10, 11, 12]]],0))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [51]:
with open('Interviewtestdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testlabels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(testlabels))

4


In [52]:
testlabels

array([1, 1, 0, 0])

In [53]:
tf.reset_default_graph()

from tensorflow.contrib import rnn 
hm_epochs = 25
batch_size = 4
state_size = 200
vocab_size = 1719
num_steps = 164

# Placeholders
context = tf.placeholder(tf.int32, [batch_size, num_steps]) # [batch_size, num_steps]
utterance = tf.placeholder(tf.int32, [batch_size,num_steps])
targets = tf.placeholder(tf.int32, [batch_size])
target = tf.placeholder(tf.int32, [batch_size])

def next_batch(step):
    p,q = x_padded[batch_size*step:batch_size*(step+1)], y_padded[batch_size*step:batch_size*(step+1)]
    t = labels[batch_size*step:batch_size*(step+1)]
    return p,q,t

def test_batch(step):
    a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    e = testlabels[batch_size*step:batch_size*(step+1)]
    return a,b,e
        
def dual_encoder_model(context,utterance,targets):
    
    #Embedding the question and answer
    context_embedded = tf.nn.embedding_lookup(embeddings, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(embeddings, utterance, name="embed_utterance")
    print(" This is the embeddings for both Q&A:", context_embedded, utterance_embedded)
    
    with tf.variable_scope("rnn") as vs:
        cell = tf.contrib.rnn.BasicLSTMCell(state_size)
        #cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)    
    
    #feed inputs to dynamic rnn for training    
    rnn_inputs = tf.concat([context_embedded, utterance_embedded],0)
    rnn_inputs = tf.transpose(rnn_inputs, [1,0,2])
    rnn_inputs = tf.reshape(rnn_inputs, [-1, state_size])
    rnn_inputs = tf.split(rnn_inputs, num_steps, 0)
    
    rnn_outputs, rnn_states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float32)
    
    encoding_context, encoding_utterance = tf.split(rnn_states.h,2,0)
    
    print("\nThis is the encoding context:", encoding_context)
    print("\nThis is the encoding utterance:", encoding_utterance)
    
    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M",
          shape=[state_size, state_size],
          initializer=tf.truncated_normal_initializer())

    # "Predict" a  response: c * M
    generated_response = tf.matmul(encoding_context, M)
    print("\nThis is the generated response:", generated_response)
    generated_response = tf.expand_dims(generated_response, 2)
    print("\nThis is the generated response:", generated_response)
    encoding_utterance = tf.expand_dims(encoding_utterance, 2)
    print("\nThis is the encoding utterance:", encoding_utterance)
    # Dot product between generated response and actual response
    # (c * M) * r
    logit = tf.matmul(generated_response, encoding_utterance, True)
    print("\nThis is the logits before squeeze:", logit)
    logits = tf.squeeze(logit, [2])
    print("\nThis is the logits after squeeze:", logits)

    # Apply sigmoid to convert logits to probabilities
    probs = tf.sigmoid(logits)   
    print("\nThis is the probability after sigmoid:", probs)
    return tf.reshape(logits, [-1]), tf.reshape(probs, [-1])

def train_encoder(x):
    prediction, probs = dual_encoder_model(context,utterance,targets)
    # Calculate the binary cross-entropy loss
    print("\nThis is the prediction size:", prediction)
    print("\nThis is the target size:", targets)
    losses = tf.nn.sigmoid_cross_entropy_with_logits(logits = prediction, labels = tf.to_float(targets))

    # Mean loss across the batch of examples
    cost = tf.reduce_mean(losses, name="cost")
    optimizer = tf.train.AdamOptimizer().minimize(cost)   
    saver = tf.train.Saver()
        
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0 
            for step in range(int(len(x_padded)/batch_size)):
                ques, ans,labl = next_batch(step)
                _, c, logts, prob = sess.run([optimizer, cost, prediction, probs], feed_dict={context: ques,
                                                              utterance: ans,  
                                                              targets: labl})
                epoch_loss += c
                #print('step:', step, 'logits:', logts, 'probability:', prob)
            print('***************************** Epoch Completed *************************************')
            print('Epoch:', epoch, 'completed out of:',hm_epochs,'loss:',epoch_loss, 'probability:', prob)
        save_path = saver.save(sess, "./Interviewmodel/")
        print("Model saved in file: %s" % save_path)  
        qe, an,lbl = test_batch(0)
        probas = sess.run(probs, feed_dict={context: qe, utterance: an, target: lbl})
        print("Probs:", probas)

In [54]:
train_encoder(x)

 This is the embeddings for both Q&A: Tensor("embed_context:0", shape=(4, 164, 200), dtype=float32) Tensor("embed_utterance:0", shape=(4, 164, 200), dtype=float32)

This is the encoding context: Tensor("split_1:0", shape=(4, 200), dtype=float32)

This is the encoding utterance: Tensor("split_1:1", shape=(4, 200), dtype=float32)

This is the generated response: Tensor("MatMul:0", shape=(4, 200), dtype=float32)

This is the generated response: Tensor("ExpandDims:0", shape=(4, 200, 1), dtype=float32)

This is the encoding utterance: Tensor("ExpandDims_1:0", shape=(4, 200, 1), dtype=float32)

This is the logits before squeeze: Tensor("MatMul_1:0", shape=(4, 1, 1), dtype=float32)

This is the logits after squeeze: Tensor("Squeeze:0", shape=(4, 1), dtype=float32)

This is the probability after sigmoid: Tensor("Sigmoid:0", shape=(4, 1), dtype=float32)

This is the prediction size: Tensor("Reshape_1:0", shape=(4,), dtype=float32)

This is the target size: Tensor("Placeholder_2:0", shape=(4,), 